<a href="https://colab.research.google.com/github/davidlealo/cft_estatales_analisis/blob/main/analisis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análisis de discurso
Instalación de API de Anthropic para el análisis de datos con la tecnología de Sonnet (específicamente claude-3-sonnet-20240229)

Lo primero es tomar la información de la clave secreta como variable de entorno a través de un input que protege la información privada de la API KEY

In [ ]:
# Instalar las bibliotecas necesarias
%pip install anthropic
%pip install PyPDF2
%pip install python-dotenv

In [16]:
import os
from dotenv import load_dotenv
from anthropic import Anthropic
from IPython.display import display, HTML

def setup_api_key():
    """Configura la API key de Anthropic desde .env o entrada manual"""

    # Intenta cargar desde .env
    load_dotenv()
    api_key = os.getenv("ANTHROPIC_API_KEY")

    if not api_key:
        # HTML para input manual
        display(HTML("""
            <div style="margin: 20px; padding: 20px; border: 1px solid #ccc;">
                <h3>Anthropic API Key</h3>
                <input type="password" id="api_key" style="width: 100%; padding: 5px;">
                <button onclick="
                    var key = document.getElementById('api_key').value;
                    google.colab.kernel.invokeFunction('notebook.store_key', [key]);
                    ">Guardar API Key</button>
            </div>
        """))

        def store_key(key):
            os.environ["ANTHROPIC_API_KEY"] = key
            return "API key almacenada"

        output.register_callback('notebook.store_key', store_key)
        return None

    return Anthropic(api_key=api_key)

# Uso:
client = setup_api_key()

In [17]:
import requests
from bs4 import BeautifulSoup
import PyPDF2
from io import BytesIO
from anthropic import Anthropic
import os

def download_and_process_documents():
    try:
        # URLs
        law_url = "https://www.bcn.cl/leychile/navegar?idNorma=1088775"
        ministry_url = "https://educacionsuperior.mineduc.cl/cft-estatales/"
        pdf_url = "https://educacionsuperior.mineduc.cl/wp-content/uploads/sites/49/2022/08/Plan-fortal-CFTE-VF.pdf"

        print("Descargando documentos...")

        # Descargar ley
        law_response = requests.get(law_url)
        law_soup = BeautifulSoup(law_response.text, 'html.parser')
        law_content = law_soup.find('div', {'id': 'normaText'})
        law_text = law_content.get_text() if law_content else "No se pudo extraer el texto de la ley"

        # Descargar información ministerial
        ministry_response = requests.get(ministry_url)
        ministry_soup = BeautifulSoup(ministry_response.text, 'html.parser')
        ministry_text = ministry_soup.get_text()

        # Descargar PDF
        pdf_response = requests.get(pdf_url)
        pdf_file = BytesIO(pdf_response.content)
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        pdf_text = "\n".join(page.extract_text() for page in pdf_reader.pages)

        print("Documentos descargados exitosamente")
        return law_text, f"{ministry_text}\n\n{pdf_text}"

    except Exception as e:
        print(f"Error al descargar documentos: {str(e)}")
        return "", ""

def analyze_cft_policy(law_text, ministry_docs):
    if not os.environ.get("ANTHROPIC_API_KEY"):
        return "Error: ANTHROPIC_API_KEY no encontrada"

    client = Anthropic(api_key=os.environ["ANTHROPIC_API_KEY"])

    system_prompt = "Analiza la Ley 20.910 sobre CFTs estatales y su implementación según documentos ministeriales."

    user_message = f"""
    Analiza los siguientes documentos:

    LEY 20.910: {law_text}
    DOCUMENTOS MINISTERIALES: {ministry_docs}

    Realiza un análisis considerando:
    1. Marco legal y objetivos establecidos en Ley 20.910
    2. Avances en implementación según Plan de Fortalecimiento
    3. Cumplimiento de hitos y metas
    4. Financiamiento y recursos asignados
    5. Articulación territorial

    Estructura obligatoria:
    - Análisis del marco normativo
    - Estado actual de implementación por región
    - Cumplimiento de objetivos legales
    - Evaluación del plan de fortalecimiento
    - Brechas y desafíos pendientes
    """

    try:
        response = client.messages.create(
            model="claude-3-sonnet-20240229",
            max_tokens=4096,
            system=system_prompt,
            messages=[{"role": "user", "content": user_message}]
        )
        return response.content[0].text
    except Exception as e:
        return f"Error en el análisis: {str(e)}"

def main():
    print("Iniciando análisis...")
    law_text, ministry_docs = download_and_process_documents()
    analysis = analyze_cft_policy(law_text, ministry_docs)

    with open("analisis_normativa_cft.txt", "w", encoding='utf-8') as f:
        f.write(analysis)
    print("Análisis completado y guardado")

if __name__ == "__main__":
    main()

Iniciando análisis...
Descargando documentos...
Documentos descargados exitosamente


<ipython-input-17-463c1e1873f0>:71: DeprecationWarning: The model 'claude-3-sonnet-20240229' is deprecated and will reach end-of-life on July 21st, 2025.
Please migrate to a newer model. Visit https://docs.anthropic.com/en/docs/resources/model-deprecations for more information.
  response = client.messages.create(


Análisis completado y guardado
